In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import numpy as np

In [2]:
sales=pd.read_csv("Sales.csv")
customers=pd.read_csv("Customers.csv",encoding='ISO_8859-1')
exchangerates=pd.read_csv("Exchange_Rates.csv")
products=pd.read_csv("Products.csv")
stores=pd.read_csv("Stores.csv")

In [3]:
sales.isnull().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [4]:
sales.dtypes

Order Number      int64
Line Item         int64
Order Date       object
Delivery Date    object
CustomerKey       int64
StoreKey          int64
ProductKey        int64
Quantity          int64
Currency Code    object
dtype: object

In [5]:
# Convert 'Order Date' to datetime format
sales['Order Date'] = pd.to_datetime(sales['Order Date'], format='%m/%d/%Y')

# Convert 'Delivery Date' to datetime format, with error coercion for invalid values
sales['Delivery Date'] = pd.to_datetime(sales['Delivery Date'], format='%m/%d/%Y', errors='coerce')

# Keep 'Currency Code' as a string, no need to convert it to datetime
sales['Currency Code'] = sales['Currency Code'].astype(str)


In [6]:
sales.fillna('Not Available',inplace=True)
sales

C:\Users\TmC\AppData\Local\Temp\ipykernel_12772\2411750818.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not Available' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  sales.fillna('Not Available',inplace=True)


,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,2016-01-01,Not Available,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13 00:00:00,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13 00:00:00,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12 00:00:00,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12 00:00:00,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,Not Available,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,2021-02-24 00:00:00,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,2021-02-23 00:00:00,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,2021-02-23 00:00:00,331277,0,1717,2,CAD


In [7]:
# Fill NaN values for 'Order Date' and 'Delivery Date' columns with pd.NaT for datetime columns
sales['Order Date'] = sales['Order Date'].fillna(pd.NaT)
sales['Delivery Date'] = sales['Delivery Date'].fillna(pd.NaT)

# Fill NaN values for non-datetime columns with 'Not Available'
non_datetime_cols = sales.select_dtypes(exclude=['datetime']).columns
sales[non_datetime_cols] = sales[non_datetime_cols].fillna('Not Available')

# Display the modified DataFrame
sales


,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,2016-01-01,Not Available,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13 00:00:00,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13 00:00:00,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12 00:00:00,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12 00:00:00,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,Not Available,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,2021-02-24 00:00:00,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,2021-02-23 00:00:00,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,2021-02-23 00:00:00,331277,0,1717,2,CAD


In [8]:
stores

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [9]:
stores.isnull().sum()

StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64

In [10]:
stores.dtypes

StoreKey           int64
Country           object
State             object
Square Meters    float64
Open Date         object
dtype: object

In [11]:
stores.fillna(0,inplace=True)

In [12]:
stores.dtypes

StoreKey           int64
Country           object
State             object
Square Meters    float64
Open Date         object
dtype: object

In [13]:
stores['Country']=stores['Country'].astype(str)
stores['State']=stores['State'].astype(str)
stores['Open Date']=pd.to_datetime(stores['Open Date'],format='%m/%d/%Y')

In [14]:
stores

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,2008-01-01
1,2,Australia,Northern Territory,665.0,2008-01-12
2,3,Australia,South Australia,2000.0,2012-01-07
3,4,Australia,Tasmania,2000.0,2010-01-01
4,5,Australia,Victoria,2000.0,2015-12-09
...,...,...,...,...,...
62,63,United States,Utah,2000.0,2008-03-06
63,64,United States,Washington DC,1330.0,2010-01-01
64,65,United States,West Virginia,1785.0,2012-01-01
65,66,United States,Wyoming,840.0,2014-01-01


In [15]:
products.isnull().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [16]:
products.dtypes

ProductKey         int64
Product Name      object
Brand             object
Color             object
Unit Cost USD     object
Unit Price USD    object
SubcategoryKey     int64
Subcategory       object
CategoryKey        int64
Category          object
dtype: object

In [17]:
products

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones


In [18]:
products.columns = products.columns.str.strip()


In [19]:
products['Unit Cost USD'] = products['Unit Cost USD'].replace(r'[\$,]', '', regex=True).astype(float)
products['Unit Price USD'] = products['Unit Price USD'].replace(r'[\$,]', '', regex=True).astype(float)


In [20]:
products.dtypes

ProductKey          int64
Product Name       object
Brand              object
Color              object
Unit Cost USD     float64
Unit Price USD    float64
SubcategoryKey      int64
Subcategory        object
CategoryKey         int64
Category           object
dtype: object

In [21]:
products

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones


In [22]:
customers.isnull().sum()


CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [23]:
customers.dtypes

CustomerKey     int64
Gender         object
Name           object
City           object
State Code     object
State          object
Zip Code       object
Country        object
Continent      object
Birthday       object
dtype: object

In [24]:
import pandas as pd

def convert_date(date_str):
    try:
        date_obj = pd.to_datetime(date_str, format='%m/%d/%Y', errors='coerce')
        if pd.notnull(date_obj):
            return date_obj.strftime('%m/%d/%Y')
        else:
            return date_str
    except ValueError:
        return date_str

customers['Birthday'] = customers['Birthday'].apply(convert_date)


In [25]:
customers['State Code'].fillna('Not Available',inplace=True)

C:\Users\TmC\AppData\Local\Temp\ipykernel_12772\115889270.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers['State Code'].fillna('Not Available',inplace=True)


In [26]:
customers['Birthday'] = pd.to_datetime(customers['Birthday'],format='%m/%d/%Y',errors='coerce')

In [27]:
customers

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,1936-03-25
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-11-09
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-10-12


In [28]:
exchangerates.dtypes

Date         object
Currency     object
Exchange    float64
dtype: object

In [29]:
exchangerates

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415
...,...,...,...
11210,2/20/2021,USD,1.0000
11211,2/20/2021,CAD,1.2610
11212,2/20/2021,AUD,1.2723
11213,2/20/2021,EUR,0.8238


In [30]:
exchangerates['Date']=pd.to_datetime(exchangerates['Date'], format='%m/%d/%Y',errors='coerce')

In [31]:
exchangerates

,Date,Currency,Exchange
0,2015-01-01,USD,1.0000
1,2015-01-01,CAD,1.1583
2,2015-01-01,AUD,1.2214
3,2015-01-01,EUR,0.8237
4,2015-01-01,GBP,0.6415
...,...,...,...
11210,2021-02-20,USD,1.0000
11211,2021-02-20,CAD,1.2610
11212,2021-02-20,AUD,1.2723
11213,2021-02-20,EUR,0.8238


In [32]:
sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,2016-01-01,Not Available,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13 00:00:00,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13 00:00:00,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12 00:00:00,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12 00:00:00,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,Not Available,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,2021-02-24 00:00:00,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,2021-02-23 00:00:00,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,2021-02-23 00:00:00,331277,0,1717,2,CAD


In [33]:
customers

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,1936-03-25
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-11-09
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-10-12


In [34]:
overall=sales

In [35]:
overall=pd.merge(overall,customers,on='CustomerKey',how='inner')

In [36]:
overall.dropna(inplace=True)

In [37]:
overall

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,366000,1,2016-01-01,Not Available,265598,10,1304,1,CAD,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,366001,1,2016-01-01,2016-01-13 00:00:00,1269051,0,1048,2,USD,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
2,366001,2,2016-01-01,2016-01-13 00:00:00,1269051,0,2007,1,USD,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
3,366002,1,2016-01-01,2016-01-12 00:00:00,266019,0,1106,7,CAD,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
4,366002,2,2016-01-01,2016-01-12 00:00:00,266019,0,373,1,CAD,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,Not Available,1216913,43,632,3,USD,Male,Donald Woodie,Garden Grove,CA,California,92643,United States,North America,1939-04-16
62880,2243031,1,2021-02-20,2021-02-24 00:00:00,511229,0,98,4,EUR,Female,Gabriele Thalberg,Stuttgart Frauenkopf,BW,Baden-Württemberg,70184,Germany,Europe,1940-03-10
62881,2243032,1,2021-02-20,2021-02-23 00:00:00,331277,0,1613,2,CAD,Male,William Rochelle,Calgary,AB,Alberta,T2T 0C6,Canada,North America,1993-05-25
62882,2243032,2,2021-02-20,2021-02-23 00:00:00,331277,0,1717,2,CAD,Male,William Rochelle,Calgary,AB,Alberta,T2T 0C6,Canada,North America,1993-05-25


In [38]:
overall=pd.merge(overall,exchangerates,left_on=['Order Date','Currency Code'],right_on=['Date','Currency'],how='inner')

In [39]:
print(overall.columns)
print(products.columns)


Index(['Order Number', 'Line Item', 'Order Date', 'Delivery Date',
       'CustomerKey', 'StoreKey', 'ProductKey', 'Quantity', 'Currency Code',
       'Gender', 'Name', 'City', 'State Code', 'State', 'Zip Code', 'Country',
       'Continent', 'Birthday', 'Date', 'Currency', 'Exchange'],
      dtype='object')
Index(['ProductKey', 'Product Name', 'Brand', 'Color', 'Unit Cost USD',
       'Unit Price USD', 'SubcategoryKey', 'Subcategory', 'CategoryKey',
       'Category'],
      dtype='object')


In [40]:
overall = pd.merge(overall, products, on='ProductKey', how='inner')


In [41]:
overall=pd.merge(overall,stores,on=['StoreKey','Country','State'],how='inner')

In [42]:
overall

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,Gender,...,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category,Square Meters,Open Date
0,366005,1,2016-01-01,Not Available,844003,33,421,4,EUR,Male,...,Adventure Works,Silver,215.68,469.00,303,Desktops,3,Computers,1540.0,2015-12-09
1,367006,1,2016-01-02,Not Available,693285,18,2445,3,EUR,Female,...,Litware,Green,2.54,4.99,808,Fans,8,Home Appliances,310.0,2012-08-08
2,367010,1,2016-01-02,Not Available,1322959,66,917,1,USD,Male,...,Southridge Video,White,59.31,179.00,308,Computers Accessories,3,Computers,840.0,2014-01-01
3,367010,2,2016-01-02,Not Available,1322959,66,39,1,USD,Male,...,Contoso,Green,99.14,299.23,101,MP4&MP3,1,Audio,840.0,2014-01-01
4,369007,1,2016-01-04,Not Available,571622,20,2510,3,EUR,Male,...,Contoso,White,2.07,4.06,505,Cell phones Accessories,5,Cell phones,1715.0,2012-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6176,2243014,1,2021-02-20,Not Available,1453427,50,423,1,USD,Male,...,Adventure Works,Black,275.46,599.00,303,Desktops,3,Computers,2000.0,2008-03-06
6177,2243014,2,2021-02-20,Not Available,1453427,50,1617,1,USD,Male,...,Contoso,Silver,26.67,57.99,602,Movie DVD,6,"Music, Movies and Audio Books",2000.0,2008-03-06
6178,2243014,3,2021-02-20,Not Available,1453427,50,2506,4,USD,Male,...,Contoso,Pink,2.42,4.74,505,Cell phones Accessories,5,Cell phones,2000.0,2008-03-06
6179,2243021,1,2021-02-20,Not Available,1783631,55,1147,4,USD,Male,...,Fabrikam,Black,301.21,655.00,405,Camcorders,4,Cameras and camcorders,2000.0,2009-12-15


In [43]:
col=list(overall.columns)
col1=[]
for c in col:
    col1.append(c.replace(' ','_').lower())
overall.columns=col1

In [44]:
col=list(stores.columns)
col1=[]
for c in col:
    col1.append(c.replace(' ','_').lower())
stores.columns=col1

In [45]:
col1

['storekey', 'country', 'state', 'square_meters', 'open_date']

In [46]:
overall.columns

Index(['order_number', 'line_item', 'order_date', 'delivery_date',
       'customerkey', 'storekey', 'productkey', 'quantity', 'currency_code',
       'gender', 'name', 'city', 'state_code', 'state', 'zip_code', 'country',
       'continent', 'birthday', 'date', 'currency', 'exchange', 'product_name',
       'brand', 'color', 'unit_cost_usd', 'unit_price_usd', 'subcategorykey',
       'subcategory', 'categorykey', 'category', 'square_meters', 'open_date'],
      dtype='object')

In [47]:
overall.isnull().sum()

order_number      0
line_item         0
order_date        0
delivery_date     0
customerkey       0
storekey          0
productkey        0
quantity          0
currency_code     0
gender            0
name              0
city              0
state_code        0
state             0
zip_code          0
country           0
continent         0
birthday          0
date              0
currency          0
exchange          0
product_name      0
brand             0
color             0
unit_cost_usd     0
unit_price_usd    0
subcategorykey    0
subcategory       0
categorykey       0
category          0
square_meters     0
open_date         0
dtype: int64

In [48]:
overall.dtypes

order_number               int64
line_item                  int64
order_date        datetime64[ns]
delivery_date             object
customerkey                int64
storekey                   int64
productkey                 int64
quantity                   int64
currency_code             object
gender                    object
name                      object
city                      object
state_code                object
state                     object
zip_code                  object
country                   object
continent                 object
birthday          datetime64[ns]
date              datetime64[ns]
currency                  object
exchange                 float64
product_name              object
brand                     object
color                     object
unit_cost_usd            float64
unit_price_usd           float64
subcategorykey             int64
subcategory               object
categorykey                int64
category                  object
square_met

In [49]:
overall[['order_date','date','currency_code','currency','customerkey']]

,order_date,date,currency_code,currency,customerkey
0,2016-01-01,2016-01-01,EUR,EUR,844003
1,2016-01-02,2016-01-02,EUR,EUR,693285
2,2016-01-02,2016-01-02,USD,USD,1322959
3,2016-01-02,2016-01-02,USD,USD,1322959
4,2016-01-04,2016-01-04,EUR,EUR,571622
...,...,...,...,...,...
6176,2021-02-20,2021-02-20,USD,USD,1453427
6177,2021-02-20,2021-02-20,USD,USD,1453427
6178,2021-02-20,2021-02-20,USD,USD,1453427
6179,2021-02-20,2021-02-20,USD,USD,1783631


In [50]:
overall.to_csv('overall_csv')

In [51]:
import pandas as pd
from sqlalchemy import create_engine

# SQL connection details
host = "localhost"
password = "svidhya"
username = "root"
port = "3306"
database = "customer_sales"

# Creating engine string
engine_string = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)

# Importing 'overall' table
table_name = "overall"
overall.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"Successfully imported {table_name} to SQL.")

# Importing 'customers' table
table_name = "customers"
customers.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"Successfully imported {table_name} to SQL.")

# Importing 'products' table
table_name = "products"
products.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"Successfully imported {table_name} to SQL.")

# Importing 'exchangerates' table
table_name = "exchangerates"
exchangerates.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"Successfully imported {table_name} to SQL.")

# Importing 'stores' table
table_name = "stores"
stores.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"Successfully imported {table_name} to SQL.")


Successfully imported overall to SQL.
Successfully imported customers to SQL.
Successfully imported products to SQL.
Successfully imported exchangerates to SQL.
Successfully imported stores to SQL.
